In [ ]:
import os
import pathlib
import numpy as np
import tensorflow as tf

In [ ]:
# Load the model from the saved Keras file
loaded_model_1 = tf.keras.models.load_model("Models/model_1.keras")

## Using Camera as model's input

In [ ]:
!ls -ltrh /dev/video* # linux command to list available cameras

#### Create the camera object
Remember to change capture_device if incorrect for your system (based on out put of camera listing /dev/video\<camera_no>)

In [ ]:
from jetcam.usb_camera import USBCamera
camera = USBCamera(width=224, height=224, capture_width=640, capture_height=480, capture_fps=10, capture_device=0)

#### Video Widget 
Create image widget object for inside-jupyter video display

In [ ]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_widet = ipywidgets.Image(format='jpeg')
image_widget.value = bgr8_to_jpeg(iamge)
camera.running = True
display(image_widget)

### Video update callback

Define callback for camera. Whenever there's an image update from camera (new frame incoming) this callback gets called. Every frame we fetch the image, rescale it to shape required by our model (if needed) and run a prediction. The prediction is diplayed in the bottom left corner of the video.

In [ ]:
import cv2

model_input_shape = (224,224)
        

def update_image(change):
    image = change['new'] # get the frame

    # Resize to model's required shape
    if(image.shape[0:2] != model_input_shape):
        image = cv2.resize(image, model_input_shape, interpolation = cv2.INTER_AREA) 
    
    # Use the model to predict a class
    predictions = loaded_model_1.predict(np.array([image]))
    pred_index = np.argmax(predictions)
    
    # Add predicted class information to the image
    cv2.putText(image, class_names[pred_index], (10, image.shape[0] - 10), font, 3, (0, 255, 0), 2, cv2.LINE_AA, False)

    # Update video widget
    image_widget.value = bgr8_to_jpeg(image)

In [ ]:
camera.observe(update_image, names='value') # subscribe to camera input

In [ ]:
camera.unobserve(update_image, names='value') # unsubscribe from camera input